# Demonstration of NN-Equivalence Tool #

In this notebook, we run through an example of verifying equivalence for two neural networks.

In [1]:
from performance import Encoder

Example neural networks are stored in the `ExampleNNs` directory.
We use the balance scale network, because they are very small and therefore their encodings are not too long, so we can print them.

In this case the test network is a version of the reference network with the value of few weights being manually altered.

In [2]:
reference = 'ExampleNNs/balance_scale_lin.h5'
test = 'ExampleNNs/balance_scale_lin2.h5'

## Verifying Top-2-Equivalence

We want to verify top-2-equivalence.

The mode is named `one_hot_partial_top_2`, because we use a partial permutation matrix to retrieve activations of the 2 most likely classes from the test network.
For the reference network, the partial permutation matrix needed to retrieve only the top output has only one row and is therefore a one hot vector.

For mode-names corresponding to top-k-equivalence for other values of k, the number at the end of the mode-name can be changed.

For $\varepsilon$-equivalence, modes `optimize_diff_[manhattan | chebyshev]` can be used.

In [3]:
mode = 'one_hot_partial_top_2'

We need to specify lower and upper bounds on the inputs.

In [4]:
input_los = [0,0,0,0]
input_his = [5,5,5,5]

Instantiate an `Encoder` object and tell it to encode equivalence between the reference and the test network.

I did not remove the debug output for the cell below. It can be ignored.

In [5]:
enc = Encoder()
enc.encode_equiv(reference, test, input_los, input_his, mode)

### Vars ###
### Constraints ###
A_pi_3_0 = 0 --> (A_x_2_0 + 0.0001) <= A_top_3_0
A_pi_3_1 = 0 --> (A_x_2_1 + 0.0001) <= A_top_3_0
A_pi_3_2 = 0 --> (A_x_2_2 + 0.0001) <= A_top_3_0


We want to restrict the input-domain for the equivalence verification to points within a radius of 5, measured in the manhattan distance.

In [6]:
center = [5,5,0,0]
radius = 5

enc.add_input_radius(center, radius, metric='manhattan')

We print the encoding of this verification instance.

The first section of the output contains the variables of the encoding with intervals describing their lower and upper bounds.
Note, that all variables other than the input variables are set to default bounds, as we didn't perform bounds optimization yet.

Every variable has a (sometimes empty) prefix, indicating, to which network it belongs.
* Variables belonging to the input layer are only encoded once and don't have a net-prefix.
* Variables belonging to the reference network are prefixed with 'A'
* Variables belonging to the test network are prefixed with 'B'
* Variables belonging to the encoding of the equivalence property are prefixed with 'E'

Furthermore every variable has two indices.
Variable `A_x_0_3` for example belongs to the first (the 0th) layer of the reference network and is in the 4th row of that layer.
For variables not belonging to layered architecture of a network, the indices don't have a special meaning.

Variables named `i` are input variables, while variables named `x` typically represent linear combinations of values of neurons in the preceding layer.
Variables named `d` represent binary variables.

Later we are interested in maximizing the variable `E_diff_0_2`, which contains the violation value for our equivalence property.

Constraints can be found in the second section of the output.

In [7]:
enc.pretty_print()

### Vars ###
i_0_0: [0, 5]
i_0_1: [0, 5]
i_0_2: [0, 5]
i_0_3: [0, 5]
d_0_0: [0, 1]
d_0_1: [0, 1]
d_0_2: [0, 1]
d_0_3: [0, 1]
abs_0_0: [0, 999999]
abs_0_1: [0, 999999]
abs_0_2: [0, 999999]
abs_0_3: [0, 999999]
A_x_0_0: [-999999, 999999]
A_x_0_1: [-999999, 999999]
A_x_0_2: [-999999, 999999]
A_x_0_3: [-999999, 999999]
A_d_0_0: [0, 1]
A_d_0_1: [0, 1]
A_d_0_2: [0, 1]
A_d_0_3: [0, 1]
A_o_0_0: [0, 999999]
A_o_0_1: [0, 999999]
A_o_0_2: [0, 999999]
A_o_0_3: [0, 999999]
A_x_1_0: [-999999, 999999]
A_x_1_1: [-999999, 999999]
A_x_1_2: [-999999, 999999]
A_x_1_3: [-999999, 999999]
A_d_1_0: [0, 1]
A_d_1_1: [0, 1]
A_d_1_2: [0, 1]
A_d_1_3: [0, 1]
A_o_1_0: [0, 999999]
A_o_1_1: [0, 999999]
A_o_1_2: [0, 999999]
A_o_1_3: [0, 999999]
A_x_2_0: [-999999, 999999]
A_x_2_1: [-999999, 999999]
A_x_2_2: [-999999, 999999]
A_y_0_0: [-999999, 999999]
A_y_1_0: [-999999, 999999]
A_y_2_0: [-999999, 999999]
A_top_3_0: [-999999, 999999]
A_pi_3_0: [0, 1]
A_pi_3_1: [0, 1]
A_pi_3_2: [0, 1]
B_x_0_0: [-999999, 999999]
B_x_0_1: [

Now we perform bounds tightening by interval arithmetic and by solving smaller sub-problems.

The output is the Gurobi-log for the subproblems solved.

In [8]:
enc.optimize_constraints()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-27
Set parameter TimeLimit to value 20
Set parameter TimeLimit to value 20
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 29 rows, 17 columns and 73 nonzeros
Model fingerprint: 0xda8fd283
Variable types: 13 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [2e-02, 7e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 7e+00]
  RHS range        [2e-02, 7e+00]
Presolve removed 9 rows and 1 columns
Presolve time: 0.00s
Presolved: 20 rows, 16 columns, 56 nonzeros
Variable types: 12 continuous, 4 integer (4 binary)
Found heuristic solution: objective 4.5258605

Root relaxation: objective 5.759716e+00, 6 iterations, 0.00 seconds (0.00 work units)

   

We can see, that the bounds of the variables have been tightened.

In [9]:
enc.pretty_print()

### Vars ###
i_0_0: [0, 5]
i_0_1: [0, 5]
i_0_2: [0, 5]
i_0_3: [0, 5]
d_0_0: [0, 0]
d_0_1: [0, 0]
d_0_2: [0, 1]
d_0_3: [0, 1]
abs_0_0: [0, 5]
abs_0_1: [0, 5]
abs_0_2: [0, 5]
abs_0_3: [0, 5]
A_x_0_0: [-4.464432448148727, 2.4794387072324753]
A_x_0_1: [-3.687929444015026, 5.7785393968224525]
A_x_0_2: [-1.7777611389756203, 3.729954317212105]
A_x_0_3: [-1.7894070968031883, 7.332582697272301]
A_d_0_0: [0, 1]
A_d_0_1: [0, 1]
A_d_0_2: [0, 1]
A_d_0_3: [0, 1]
A_o_0_0: [0, 2.4794387072324753]
A_o_0_1: [0, 5.7785393968224525]
A_o_0_2: [0, 3.729954317212105]
A_o_0_3: [0, 7.332582697272301]
A_x_1_0: [-1.8478958275373936, 5.076723421703085]
A_x_1_1: [-0.32485962698912885, 1.8436871195505793]
A_x_1_2: [-1.804407192748503, 8.159865636119198]
A_x_1_3: [-6.631261264248645, 4.4843847549760945]
A_d_1_0: [0, 1]
A_d_1_1: [0, 1]
A_d_1_2: [0, 1]
A_d_1_3: [0, 1]
A_o_1_0: [0, 5.076723421703085]
A_o_1_1: [0, 1.8436871195505793]
A_o_1_2: [0, 8.159865636119198]
A_o_1_3: [0, 4.4843847549760945]
A_x_2_0: [-6.915419027

Now we use Gurobi to optimize the encoding.

At the very top of the log, we can see the size of the encoding.
A few lines below is the size of the mixed integer linear program (MILP) after application of Gurobi's presolve routine.

The column `Incumbent` contains the best (in this case the maximal) value of the objective function (here the variable `E_diff_0_2` representing the violation score of top-2-equivalence).
The column `BestBd` contains bounds on this value (upper bounds for maximization problems, lower bounds for minimization problems).

The final solution is printed at the bottom.
As it is greater than zero, the test network is not top-2-equivalent to the reference network.

In [10]:
model = enc.create_gurobi_model()
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 187 rows, 95 columns and 484 nonzeros
Model fingerprint: 0xa7e4f7c0
Variable types: 63 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 2e+01]
  RHS range        [1e-04, 2e+01]
Presolve removed 59 rows and 21 columns
Presolve time: 0.00s
Presolved: 128 rows, 74 columns, 404 nonzeros
Variable types: 52 continuous, 22 integer (22 binary)

Root relaxation: objective 1.121046e+01, 69 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   11.21046    0   15          -   11.

In [11]:
from analysis import get_grb_inputs, calculate_violation

We can now analyze the counterexample found by Gurobi.

First we retrieve the counterexample. The required method has signature `get_grb_inputs(model, input_size)`.

In [12]:
ins = get_grb_inputs(model, 4)
print(ins)

[5.0, 5.0, 5.0, 0.0]


We have a dedicated method to calculate the actual violation to top-k-equivalence.

It takes the found counterexample, the value of k and the paths to the verified neural networks as inputs.

In [13]:
calculate_violation(ins, reference, test, top_k=2)

2.9512734022428213

## Maximizing Equivalence Radius

This time we want to find a maximal equivalence radius around our centerpoint. First we instantiate a new encoder.

In [14]:
enc_r = Encoder()
enc_r.encode_equiv(reference, test, input_los, input_his, mode)

### Vars ###
### Constraints ###
A_pi_3_0 = 0 --> (A_x_2_0 + 0.0001) <= A_top_3_0
A_pi_3_1 = 0 --> (A_x_2_1 + 0.0001) <= A_top_3_0
A_pi_3_2 = 0 --> (A_x_2_2 + 0.0001) <= A_top_3_0


Then we add an input radius, however with `radius_mode='variable'`.

We use an upper bound of 5 on our radius, since we know, that they are not equivalent for this radius from our first verification problem.

In [15]:
radius_lo = 0
radius_hi = 5

enc_r.add_input_radius(center, radius_hi, metric='manhattan', radius_lo=radius_lo, radius_mode='variable')

We can then optimize for the radius.

We omit the bounds tightening here, since the balance_scale networks are very small.

In [16]:
model_r = enc_r.create_gurobi_model()
model_r.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 205 rows, 96 columns and 528 nonzeros
Model fingerprint: 0xe8e4a876
Variable types: 64 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e-04, 2e+06]
Presolve removed 74 rows and 22 columns
Presolve time: 0.01s
Presolved: 131 rows, 74 columns, 412 nonzeros
Variable types: 51 continuous, 23 integer (23 binary)

Root relaxation: objective 0.000000e+00, 52 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    4          -    0.

First, we test, if Gurobi found a valid counterexample for radius 1.99797.

As it produces a violation greater than 0.01, it is exceeds our threshold for equivalence and thus is a valid counterexample.

The equivalence threshold, along with other flags and constants can be found in the `flags_constants.py` module.

In [17]:
ins_r = get_grb_inputs(model_r, 4)
print('Counterexample: ', ins_r)

calculate_violation(ins_r, reference, test, top_k=2)

Counterexample:  [5.0, 5.0, 1.9979682622002195, 0.0]


0.009999999989681374

Now we test, if Gurobi finds, that the two networks are equivalent, when restricting the input-region to a radius just below the smallest radius, for which Gurobi still found a counterexample, whose violation was larger than 0.01.

Within a radius of 1.95, the violation of the equivalence property is indeed almost zero.

In [18]:
enc_t = Encoder()
enc_t.encode_equiv(reference, test, input_los, input_his, mode)
enc_t.add_input_radius(center, 1.95)
model_t = enc_t.create_gurobi_model()
model_t.optimize()

### Vars ###
### Constraints ###
A_pi_3_0 = 0 --> (A_x_2_0 + 0.0001) <= A_top_3_0
A_pi_3_1 = 0 --> (A_x_2_1 + 0.0001) <= A_top_3_0
A_pi_3_2 = 0 --> (A_x_2_2 + 0.0001) <= A_top_3_0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 204 rows, 95 columns and 526 nonzeros
Model fingerprint: 0x6d67559c
Variable types: 63 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e-04, 2e+06]
Presolve removed 141 rows and 56 columns
Presolve time: 0.00s
Presolved: 63 rows, 39 columns, 200 nonzeros
Variable types: 26 continuous, 13 integer (13 binary)

Root relaxation: objective 4.536788e+00, 38 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Cur

## Verifying $\epsilon$-Equivalence

To verify $\epsilon$-equivalence, we follow the same pattern as for top-$2$-equivalence above.

First, we specify the mode.

`optimize_diff_manhattan` checks for equivalence using $\| \mathcal{R}(x) - \mathcal{T}(x) \|_1$, while `optimize_diff_chebyshev` checks for equivalence in the sense of $\| \mathcal{R}(x) - \mathcal{T}(x) \|_\infty$.

Below, we are going to check for equivalence in the Chebyshev norm.

In [19]:
mode = 'optimize_diff_chebyshev'

Next, we specify the valid input range.

In [20]:
input_los = [0,0,0,0]
input_his = [5,5,5,5]

Then, we add the output constraints.

In [21]:
enc = Encoder()
enc.encode_equiv(reference, test, input_los, input_his, mode)

Since we don't want to verify a propertey over the entire region of valid inputs, but only a subset, we can specify a radius in either the Manhattan or Chebyshev norm around some center input.

(Considering the Chebyshev norm, we could just set `input_los` and `input_his` to the corresponding values, but for the Manhattan norm, this would not be possible.)

In [22]:
center = [5,5,0,0]
radius = 5

enc.add_input_radius(center, radius, metric='chebyshev')

Since we need to check for the largest element-wise absolute difference in the output of the reference and the test network, we compute the absolute differences (the `E_abs_d_0_i` variables) and use a partial permutation matrix, to ensure that `E_o_1_0` contains the largest absolute difference.

In [23]:
enc.pretty_print()

### Vars ###
i_0_0: [0, 5]
i_0_1: [0, 5]
i_0_2: [0, 5]
i_0_3: [0, 5]
A_x_0_0: [-999999, 999999]
A_x_0_1: [-999999, 999999]
A_x_0_2: [-999999, 999999]
A_x_0_3: [-999999, 999999]
A_d_0_0: [0, 1]
A_d_0_1: [0, 1]
A_d_0_2: [0, 1]
A_d_0_3: [0, 1]
A_o_0_0: [0, 999999]
A_o_0_1: [0, 999999]
A_o_0_2: [0, 999999]
A_o_0_3: [0, 999999]
A_x_1_0: [-999999, 999999]
A_x_1_1: [-999999, 999999]
A_x_1_2: [-999999, 999999]
A_x_1_3: [-999999, 999999]
A_d_1_0: [0, 1]
A_d_1_1: [0, 1]
A_d_1_2: [0, 1]
A_d_1_3: [0, 1]
A_o_1_0: [0, 999999]
A_o_1_1: [0, 999999]
A_o_1_2: [0, 999999]
A_o_1_3: [0, 999999]
A_x_2_0: [-999999, 999999]
A_x_2_1: [-999999, 999999]
A_x_2_2: [-999999, 999999]
B_x_0_0: [-999999, 999999]
B_x_0_1: [-999999, 999999]
B_x_0_2: [-999999, 999999]
B_x_0_3: [-999999, 999999]
B_d_0_0: [0, 1]
B_d_0_1: [0, 1]
B_d_0_2: [0, 1]
B_d_0_3: [0, 1]
B_o_0_0: [0, 999999]
B_o_0_1: [0, 999999]
B_o_0_2: [0, 999999]
B_o_0_3: [0, 999999]
B_x_1_0: [-999999, 999999]
B_x_1_1: [-999999, 999999]
B_x_1_2: [-999999, 999999]
B

If we wanted to, we could again tighten the bounds using `enc.optimize_constraints()`,
however, for a problem of small size, like the current one, we skip this step.

Instead, we directly optimize the model.

In [24]:
model = enc.create_gurobi_model()
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 157 rows, 78 columns and 398 nonzeros
Model fingerprint: 0x0ce91941
Variable types: 53 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [2e-02, 2e+06]
Presolve removed 45 rows and 11 columns
Presolve time: 0.00s
Presolved: 112 rows, 67 columns, 326 nonzeros
Variable types: 46 continuous, 21 integer (21 binary)

Root relaxation: objective 2.725214e+01, 47 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   27.25214    0   10          -   27.

Gurobi solved the model to optimality and claims that the maximum Chebyshev distance between the reference and the test network is $9.24$.

In [28]:
from analysis import compare_outputs, calculate_distance

We obtain the maximizing input:

In [26]:
ins = get_grb_inputs(model, 4)
print(ins)

[5.0, 5.0, 5.0, 2.02225504080907]


With a quick check, we can confirm that indeed, the difference in the first output leads to a distance of $9.24$.

In [27]:
compare_outputs(reference, test, ins)

+--------+-------------------------+------------------------+
| Neuron |          NN 1           |          NN 2          |
+========+=========================+========================+
| 0      | 6.669                   | (- 2.5716000255475517) |
+--------+-------------------------+------------------------+
| 1      | 1.138                   | 1.518                  |
+--------+-------------------------+------------------------+
| 2      | (- 0.35190319897864414) | 6.769                  |
+--------+-------------------------+------------------------+


In [30]:
calculate_distance(ins, reference, test, mode='chebyshev')

9.240681994240212

If we are only interested in checking whether $\| \mathcal{R}(x) - \mathcal{T}(x) \|_p \leq \delta$ for $p \in \{1, \infty\}$ and some constant $\delta$, we can also run the optimization as above, but stop execution as soon as either
- an input was found that yields a larger distance in the specified norm or
- the obtained upper bound on the distance is already $\leq \delta$.

This can be achieved via

In [57]:
delta = 20

model = enc.create_gurobi_model()
model.setParam('BestObjStop', delta)
model.setParam('BestBdStop', delta)
model.optimize()

Set parameter BestObjStop to value 20
Set parameter BestBdStop to value 20
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 157 rows, 78 columns and 398 nonzeros
Model fingerprint: 0x0ce91941
Variable types: 53 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [2e-02, 2e+06]
Presolve removed 45 rows and 11 columns
Presolve time: 0.00s
Presolved: 112 rows, 67 columns, 326 nonzeros
Variable types: 46 continuous, 21 integer (21 binary)

Root relaxation: objective 2.725214e+01, 47 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Best

In this setting, the optimization was stopped, because the current upper bound of $14.66$ is smaller than the specified value of $\delta = 20$.